# Selenium을 이용하여 유튜브 채널의 특정 댓글 추출하기
- 2021.03.14 : 댓글(comment) 크롤링 구현
- 2021.03.15 : 답글(reply) 크롤링 구현
- 2021.03.16 : 답글 크롤링 디버그 + 댓글/답글 '자세히 보기' 클릭 구현

### 필요한 라이브러리 불러오기

In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains

### 설정값

In [2]:
# 목표 채널의 '동영상' 카테고리 url
channel = "https://www.youtube.com/channel/UCwkGvF7xKz2E0Lv-fZ9wv2g/videos"

# 콘텐츠 로딩 대기 시간
long_waiting = 3.0
short_waiting = 1.0

### 원하는 유튜브 채널의 '동영상' 카테고리에서 모든 영상 url 크롤링

In [3]:
driver = webdriver.Chrome('chromedriver.exe')
driver.get(channel)
href = []
# 현재 화면의 높이를 구한다.
last_height = driver.execute_script("return document.documentElement.scrollHeight")
while True:
    # 스크롤을 맨 아래까지 내린다.
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

    # 콘텐츠 로딩 시간을 기다린다.
    time.sleep(short_waiting)

    # 현재 높이와 이전 높이값을 비교한다(더 스크롤할 지를 검사)
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

videos = driver.find_elements_by_id('video-title')

# 테스트 목적으로 5개의 영상만 크롤링.
num = 5
for video in videos:
    href += [video.get_attribute('href')]
    if len(href) >= num:
        break

### 각 영상에 접근하여 원하는 키워드를 포함한 댓글 크롤링

In [4]:
i = 0
# 채널의 각 영상에 접근한다.
for url in href:
    driver.get(url)
    print(f"<Access the video of {i}>")
    time.sleep(long_waiting)
    
    # 댓글창이 화면에 보일 때까지 적당히 스크롤(버퍼링 오작동 방지)
    driver.execute_script("window.scrollTo(0, 500);")
    time.sleep(long_waiting)
    
    # 모든 댓글을 크롤링 할 수 있도록 무한 스크롤
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(long_waiting)
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    # 각 영상의 원하는 댓글을 추출한다.
    comments_area = driver.find_elements_by_css_selector('#contents > ytd-comment-thread-renderer')
    find = False
    for comment_area in comments_area:
        # 각 댓글이 '자세히 보기' 버튼을 가지고 있으면 클릭.
        try:
            comment_area.find_element_by_css_selector('#more > span').click()
        except:
            pass
        comment = comment_area.find_element_by_css_selector('#comment').text
        
        # 목적에 맞는 댓글일 경우
        if "?" in comment:
            find = True
            # 댓글 출력
            print(f"Comment block : \n{comment}")
            actions = ActionChains(driver)
            actions.move_to_element(comment_area).perform()
            time.sleep(short_waiting)
            reply_area = comment_area.find_element_by_css_selector('#replies')
            # 답글이 있는 댓글인지를 검사
            try:
                button = reply_area.find_element_by_css_selector('#button > yt-icon')
                # '답글 보기' 버튼을 클릭
                button.click()
                time.sleep(long_waiting)
            except:
                pass
            # '자세히 보기' 버튼이 있는 답글을 모두 클릭
            try:
                long_texts = reply_area.find_elements_by_css_selector('#more > span')
                for long_text in long_texts:
                    long_text.click()
                    time.sleep(short_waiting)
            except:
                pass
            # 모든 답글 크롤링
            replies = reply_area.find_elements_by_css_selector('#content-text')
            for reply in replies:
                print(f"  └ {reply.text}")
    # 댓글을 한 개 이상 찾았을 경우 해당 영상의 url 출력
    if find:
        print(f"url : {url}")
    i+=1

driver.close()

<Access the video of 0>
Comment block : 
최창현
4개월 전
좋은 책 감사합니다 ㅎㅎ 고생 많으셨습니다! 혹시 팡요랩의 향후 계획도 알 수 있을까요?!
4
답글
  └ 제가 개인적인 사정으로 너무 바빠서.. 유튜브 활동을 못하고 있네요 하하..
Comment block : 
박동연
3개월 전
안녕하세요. 좋은 책 감사합니다. 덕분에 강화학습에 흥미를 느끼며 공부하고 있습니다. 다름이 아니라 책을 보며 이해가 되지 않는 부분이 있어서 질문을 하려 했는데 어디에 질문을 해야할까 고민하다가 유튜브에 질문남기게 되었습니다.

책 115페이지 메인함수에 대한 코드에서 코드 중 transition에 대한 for 반복문에 있어서 cum_reward = cum_reward + gamma*reward라는 코드가 나옵니다. 

그런데 위 코드중 우변의 cum_reward라는 식은 G_(t+1)을 뜻하고 좌변의 cum_reward는 G_t를 뜻하는 것 아닌가요? (방문했던 상태들을 뒤에서부터 보며 차례차례 리턴을 계산하기 때문에 G_또한 시점이 거꾸로 가면서 업데이트(t+1 -> t)되는 것이라고 이해하였습니다.) 

만약 제가 이해한 바가 맞다면, 이를 G_t= R_t+ γG_(t+1) 에 비추어 보았을 때 

cum_reward = cum_reward + gamma*reward 가 아닌

cum_reward = gamma*cum_reward + reward 가 되어야 하는 것이 아닌지요?

혹시 제가 잘못 이해한 부분이 있다면 바로 잡아주시면 감사하겠습니다!
간략히
답글
  └ 수정되어야 할 부분입니다! 2쇄에서는 수정되어서 출간 될 예정입니다
url : https://www.youtube.com/watch?v=Sq-JOue82QA
<Access the video of 1>
Comment block : 
Jongwon Kim
1년 전
발표자료는 공개해 주실수 없나요?
답글
Comment block : 
김평
2시간 전
먼저 좋은 강의 감사드립니다